# ParFlow Short Course: Overland Flow [SOLUTIONS]
## Exercise: Rainfall Runoff Hydrograph on a Tilted-V

### Domain Description
 - A tilted V domain that consists of two hillslopes sloping toward a central channel. 
 - The hillslopes and channel are created using three separate box geometries. 
 - The domain is consists of a single vertical layer 0.05m thick, and 5 grid cells in the x and y direction each with a cell size of 10.0
 - The domain is initialized to be totally dry (with a water table set at 3m below the surface, which is below the bottom of the domain)
 - Permeability, porosity and  hydraulic conductivity are all set very low to create an impermeable surface as we are really just focusing on runoff in this test. 
 - Simulations run for 2 hours with a time step of 0.05 hours. 
 - Rain is at a constant rate of 0.01m/hour for 0.1 hours (2 timesteps) followed by a rain free period for the rest of the simulation 
 - The examples show how to run with three different overland flow formulations: OverlandFlow, OverlandKinematic and Diffusive

### Activities
You will explore overland flow on a Tilted-V domain. Please see [tilted_v_domain_setup.ipynb](https://github.com/hydroframe/parflow_short_course_updated/blob/main/overland_flow/tilted_v_domain_setup.ipynb) if you would like to see the full set of keys used to define the domain for this ParFlow run. In this section we are focusing on the ParFlow keys related to the [Topographical Slopes](https://parflow.readthedocs.io/en/latest/keys.html#topographical-slopes), [Manning's Roughness Values](https://parflow.readthedocs.io/en/latest/keys.html#mannings-roughness-values), and [Time Cycles](https://parflow.readthedocs.io/en/latest/keys.html#time-cycles). We will explain these keys in more detail in the cells below. Also feel free to explore the [ParFlow manual](https://parflow.readthedocs.io/en/latest/keys.html#) for detail on all ParFlow keys.

Please complete the following:
1. Change the slope of the channel and the manning's roughness and see how it affects the timing of runoff
1. Add a second rainstorm by decreasing the recession length (The key for this is: overland.Cycle.rainrec.rec.Length = 300) Note that 300 is the number of timesteps so this is actually 15 hours. 
1. Turn the V so the streamflow flows out the north end of the domain instead of the south (This will also require them to change the outlet point that they plot for streamflow)
1. Extra challenge: Turn the V 90 degrees. 

In [ ]:
# Import the ParFlow package
from parflow import Run
import os
from parflow.tools.fs import mkdir, cp, chdir, get_absolute_path, rm, exists

Below are three formulations for how to set up a Tilted V domain. Please select and run one and complete the exercises with it.

In [ ]:
# Import run information from pfidb
overland = Run.from_definition("overland_tiltedV.pfidb")

#-----------------------------------------------------------------------------
# Original formulation with a zero value channel
#-----------------------------------------------------------------------------
overland.TopoSlopesX.Type = 'Constant'
overland.TopoSlopesX.GeomNames = 'left right channel'
overland.TopoSlopesX.Geom.left.Value = -0.01
overland.TopoSlopesX.Geom.right.Value = 0.01
overland.TopoSlopesX.Geom.channel.Value = 0.00

overland.TopoSlopesY.Type = 'Constant'
overland.TopoSlopesY.GeomNames = 'domain'
overland.TopoSlopesY.Geom.domain.Value = 0.01

# original approach from K&M AWR 2006
overland.Patch.z_upper.BCPressure.Type = 'OverlandFlow'
overland.Solver.Nonlinear.UseJacobian = True
overland.Solver.Linear.Preconditioner.PCMatrixType = 'PFSymmetric'

In [ ]:
# Import run information from pfidb
overland = Run.from_definition("overland_tiltedV.pfidb")

#-----------------------------------------------------------------------------
# New kinematic formulations without the zero channel
# Note: The difference in configuration here is to be consistent with the way
#   the upwinding is handled for the new and original fomulations.
#   These two results should be almost identical for the new and old formulations
#-----------------------------------------------------------------------------
overland.TopoSlopesX.Type = 'Constant'
overland.TopoSlopesX.GeomNames = 'left right channel'
overland.TopoSlopesX.Geom.left.Value = -0.01
overland.TopoSlopesX.Geom.right.Value = 0.01
overland.TopoSlopesX.Geom.channel.Value = 0.01

overland.TopoSlopesY.Type = 'Constant'
overland.TopoSlopesY.GeomNames = 'domain'
overland.TopoSlopesY.Geom.domain.Value = 0.01

# run with KWE upwinding
overland.Patch.z_upper.BCPressure.Type = 'OverlandKinematic'
overland.Solver.Nonlinear.UseJacobian = True
overland.Solver.Linear.Preconditioner.PCMatrixType = 'PFSymmetric'

In [ ]:
# Import run information from pfidb
overland = Run.from_definition("overland_tiltedV.pfidb")

#-----------------------------------------------------------------------------
# New diffusive formulations without the zero channel (as compared to the first
#    tests in overland_tiltedV_KWE.tcl)
# Note: The difference in configuration here is to be consistent with the way
#   the upwinding is handled for the new and original fomulations.
#   These two results should be almost identical for the new and old formulations
# Commented lines are from original TCL test - will need to convert to Python if running with
# Python pftools
#-----------------------------------------------------------------------------
overland.TopoSlopesX.Type = 'Constant'
overland.TopoSlopesX.GeomNames = 'left right channel'
overland.TopoSlopesX.Geom.left.Value = -0.01
overland.TopoSlopesX.Geom.right.Value = 0.01
overland.TopoSlopesX.Geom.channel.Value = 0.01

overland.TopoSlopesY.Type = 'Constant'
overland.TopoSlopesY.GeomNames = 'domain'
overland.TopoSlopesY.Geom.domain.Value = 0.01

# run with DWE
overland.Patch.z_upper.BCPressure.Type = 'OverlandDiffusive'
overland.Solver.Nonlinear.UseJacobian = True
overland.Solver.Linear.Preconditioner.PCMatrixType = 'PFSymmetric'

Now that you have selected your formulation, the next code cells relate to keys for the activities.

Activity 1 [SOLUTION]: Change the slope of the channel and the manning's roughness and see how it affects the timing of runoff

In [ ]:
#---------------------------------------------------------
# Channel slope
#---------------------------------------------------------
overland.TopoSlopesX.Geom.channel.Value = 0.03 # NOTE: updated from 0.00

#---------------------------------------------------------
# Mannings coefficient
#---------------------------------------------------------
overland.Mannings.Type = 'Constant'
overland.Mannings.GeomNames = 'domain'
overland.Mannings.Geom.domain.Value = 3.e-5 # NOTE: updated from 3.e-6

Activity 2 [SOLUTION]: Add a second rainstorm by decreasing the recession length (The key for this is: overland.Cycle.rainrec.rec.Length = 300) Note that 300 is the number of timesteps so this is actually 15 hours. 

In [ ]:
#-----------------------------------------------------------------------------
# Time Cycles
#-----------------------------------------------------------------------------
overland.Cycle.Names = 'constant rainrec'
overland.Cycle.constant.Names = 'alltime'
overland.Cycle.constant.alltime.Length = 1
overland.Cycle.constant.Repeat = -1

# rainfall and recession time periods are defined here
# rain for 1 hour, recession for 2 hours
overland.Cycle.rainrec.Names = 'rain rec'
overland.Cycle.rainrec.rain.Length = 2
overland.Cycle.rainrec.rec.Length = 30 # NOTE: updated from 300
overland.Cycle.rainrec.Repeat = -1

Activity 3 [SOLUTION]: Turn the V so the streamflow flows out the north end of the domain instead of the south (This will also require them to change the outlet point that they plot for streamflow)

In [ ]:
#---------------------------------------------------------
# Change flow direction
#---------------------------------------------------------
overland.TopoSlopesY.Geom.domain.Value = -0.01 # NOTE: updated from 0.01

Challenge [SOLUTION]: Turn the V 90 degrees. 

In [ ]:
#---------------------------------------------------------
# Challenge: Turn V 90 degrees
# (hint: changing some of the following keys will accomplish this)
#---------------------------------------------------------
# NOTE: Swapped TopoSlopesX and TopoSlopesY definitions
overland.TopoSlopesY.Type = 'Constant'
overland.TopoSlopesY.GeomNames = 'left right channel'
overland.TopoSlopesY.Geom.left.Value = -0.01
overland.TopoSlopesY.Geom.right.Value = 0.01
overland.TopoSlopesY.Geom.channel.Value = 0.00

overland.TopoSlopesX.Type = 'Constant'
overland.TopoSlopesX.GeomNames = 'domain'
overland.TopoSlopesX.Geom.domain.Value = 0.01

In [ ]:
#-----------------------------------------------------------------------------
# Run ParFlow
#-----------------------------------------------------------------------------
base = os.path.join(os.getcwd(), "output")
mkdir(base)
print(f"base: {base}")
overland.run(working_directory=base)

Now we will plot some of the ParFlow outputs to visualize the pressure and a hydrograph at the outlet point.

In [ ]:
import parflow as pf
import numpy as np  # we will use numpy to work with the data
import matplotlib.pyplot as plt  # we will use matplotlib to plot the data
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
from parflow import Run
import parflow.tools.hydrology as hydro

def plot_domain(run_directory, variable, timestep=0):
    """Function to plot output from a ParFlow run"""

    # Load the run from the file, this is the same as the run defined above
    run = Run.from_definition(os.path.join(run_directory, "overland_tiltedV.pfidb"))   

    data = run.data_accessor # get the data accessor, this makes it easier to access the data from the run
    nt = len(data.times)  # get the number of time steps
    nx = data.shape[2]    # get the number of cells in the x direction
    ny = data.shape[1]    # get the number of cells in the y direction
    nz = data.shape[0]    # get the number of cells in the z direction
    dx = data.dx          # get the cell size in the x direction
    dy = data.dy          # get the cell size in the y direction
    dz = data.dz          # get the cell size in the z direction, this is a 1D array of size nz

    # Print a summary of the run data
    print(f"nx = {nx}, ny = {ny}, nz = {nz}, nt = {nt}")
    print(f"dx = {dx}, dy = {dy}, dz = {dz}")

    # Load the data
    data = read_pfb(get_absolute_path(f"overland_tiltedV.out.{variable}.{str(timestep).zfill(5)}.pfb"))[0, :, :]
    
    # Set negative saturation values to NaN
    if variable == "satur":
        data[data < 0.0] = np.nan
    
    # Set up x and z to match the shape of the ParFlow grid
    x = np.arange(0.0,(nx+1)*dx,dx)
    y = np.arange(0.0,(ny+1)*dy,dy)
    z = np.zeros(nz+1)
    z[1:] = np.cumsum(dz)

    # Get limits for plotting
    vmin = np.nanmin(data)
    vmax = np.nanmax(data)
    print(f"vmin: {vmin}, vmax: {vmax}")
    
    # Define labels for plots
    if variable == "satur":
        label = "Saturation [-]"
        title = "Saturation"
    elif variable == "press":
        label = "Pressure Head [m]"
        title = "Pressure Head"

    # Use pcolormesh to plot the data with the x and z coordinates with lines 
    # for the grid mesh from the ParFlow run grid
    fig, ax = plt.subplots()
    im = ax.pcolormesh(x, y, data, vmin=vmin, vmax=vmax, cmap='plasma_r')
    plt.colorbar(im, ax=ax, label=label)
    
    # Include mesh lines
    ax.hlines(z,x[0],x[-1],colors='white',linewidth=0.5)
    ax.vlines(x,z[0],z[-1],colors='white',linewidth=0.5)
    
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title(f"{title} at t={timestep}")
    plt.show()

In [ ]:
# Call plotting function at multiple timesteps to see how pressure changes over time
timesteps = [0, 10, 20, 30, 40]

for t in timesteps:
    plot_domain(base, "press", timestep=t)